## Celovitist in zgoščevanje

In [ ]:
import os
from cryptography.hazmat.primitives.cmac import CMAC
from cryptography.hazmat.primitives.ciphers import algorithms
from cryptography.hazmat.primitives.hmac import HMAC, hashes

### CMAC

In [ ]:
# Funkcija za izračun CMAC podpisa
def cmac_sign(key: bytes, msg: bytes) -> bytes:
    cmac = CMAC(algorithms.AES(key))
    cmac.update(msg)
    return cmac.finalize()

# Preveritev podpisa z primerjanjem značke
def cmac_verify(key: bytes, msg: bytes, tag: bytes) -> bool:
    if cmac_sign(key, msg) == tag:
        return True
    else:
        return False

### HMAC

In [ ]:
# Funkcija za izračun HMAC podpisa
def hmac_sign(key: bytes, msg: bytes) -> bytes:
    hmac = HMAC(key, hashes.SHA256())
    hmac.update(msg)
    return hmac.finalize()

# Preveritev podpisa z primerjanjem značke
def hmac_verify(key: bytes, msg: bytes, tag: bytes) -> bool:
    if hmac_sign(key, msg) == tag:
        return True
    else:
        return False

## Javanski del

```java
package vp.integrity;

import java.nio.charset.StandardCharsets;
import java.security.MessageDigest;
import java.security.NoSuchAlgorithmException;
import java.util.Arrays;
import java.util.HexFormat;
import java.security.Key;
import javax.crypto.KeyGenerator;
import javax.crypto.Mac;
import fri.isp.Environment;
```

### Zgoščevanje sporočila z MessageDigest

**Class MessageDigest naredi zgoščevanje sporočila z uporabo zgoščevalnega algoritma SHA3-256 in izpiše rezultate v različnih oblikah.
Moznosti za zgoščevalne algoritme so na voljo v [dokumentaciji Jave](https://docs.oracle.com/en/java/javase/21/docs/specs/security/standard-names.html#messagedigest-algorithms).**

```java
public class MessageDigest {

    public static void main(String[] args) throws NoSuchAlgorithmException {
        final String message = "Moje sporočilo";
        final byte[] pt = message.getBytes(StandardCharsets.UTF_8);

        // Izberemo ustrezno zgoščevalno funkcijo. Seznam veljavnih imen algoritmov najdemo v dokumentaciji
        final MessageDigest algorithm = MessageDigest.getInstance("SHA3-256");

        algorithm.update(pt);   // dodamo čistopis
        final byte[] hashed = algorithm.digest();   // zaključimo zgoščevanje
        // Izpišemo rezultate
    }
}
```

```java
// Izpišemo rezultate
System.out.printf("Sporočilo: %s%n", new String(pt, StandardCharsets.UTF_8));   // pretvorba iz byte[] v String
System.out.printf("Zgostitev (šestnajstiško): %s%n", HexFormat.of().formatHex(hashed)); // pretvorba v šestajstiški niz
System.out.printf("Zgostitev (polje bajtov): %s%n", Arrays.toString(hashed));   // izpis kot polje bajtov
//(v javi se bajti predstavijo s predznakom)
```

### HMAC z uporabo zgoščevalne funkcije SHA3-256 (En class)

Pogovor med Ano in Borom, kjer Ana pošlje sporočilo skupaj z njegovo HMAC značko, ki jo izračuna z uporabo zgoščevalne funkcije SHA3-256 in skupnega skrivnega ključa.

```java
public class HMACExample {
    public static void main(String[] args) throws Exception {
        final String message = "Moje sporočilo.";
        final byte[] pt = message.getBytes(StandardCharsets.UTF_8);

        // ANA ustvari algoritem HMAC z uporabo SHA3-512
        final Key key = KeyGenerator.getInstance("HmacSHA3-512").generateKey(); // Ustvarimo naključen ključ
        final Mac ana = Mac.getInstance("HmacSHA3-512");    // Ustvarimo algoritem MAC

        ana.init(key);  // In ji podamo ključ
        final byte[] tag = ana.doFinal(pt); // Izračunamo značko iz sporočila

        final String tagHex = HexFormat.of().formatHex(tag);
        System.out.println("HMAC: " + tagHex);  // Izpišemo značko v šestnajstiški obliki


        // BOR prejme sporočilo in značko od Ane
        final Mac bor = Mac.getInstance("HmacSHA3-512");

        bor.init(key);  // Uporabi isti ključ kot Ana
        final byte[] recomputedTag = bor.doFinal(pt);

        // Pri preverjanju moramo paziti na napade z merjenjem časa, zato primerjamo s funkcijo MessageDigest.isEqual
        if (!MessageDigest.isEqual(tag, recomputedTag)) {   // Pteverimo značko
            System.out.println("Received invalid data, aborting!");
            System.exit(1);
        }
        System.out.printf("Data appears to be valid: '%s'%n", new String(pt, StandardCharsets.UTF_8));
    }
}
```

### Komunikacija med agenti z uporabo HMAC

To je primer simulacijskega okolja, v katerem si agenti (Ana, Bor, Cene, Nandi idr.) izmenjujejo sporočila.
Vsak agent se izvaja v svoji niti in neodvisno od ostalih. S pomočjo klicev funkcije `send(byte[])` in `byte[] receive(String)` agenti pošiljajo in prejemajo sporočila.
Enota poslanega podatka je polje bajtov `(byte[])`, zato je treba vsak podatek pred pošiljanjem postrojiti.

```java
public class CommunicationExampleHMAC {
    public static void main(String[] args) throws Exception {

        // Simulacijsko okolje, v katerem bivajo agenti
        final Environment env = new Environment();

        final Key key = KeyGenerator.getInstance("HmacSHA3-256").generateKey();

        env.add(new Agent("ana") {
            @Override
            public void task() throws Exception {
                final Mac ana = Mac.getInstance("HmacSHA3-256");
                final byte[] pt = "Zdravo Bor, tukaj Ana.".getBytes(StandardCharsets.UTF_8);

                ana.init(key);
                ana.update(pt);

                send("bor", pt);
                send("bor", ana.doFinal());
                print("Poslala sem sporočilo.");
            }
        });

        env.add(new Agent("bor") {
            @Override
            public void task() throws Exception {
                // Bor prejme sporočilo

                final byte[] pt = receive("ana");
                final byte[] tag = receive("ana");

                final Mac bor = Mac.getInstance("HmacSHA3-256");
                bor.init(key);
                bor.update(pt);
                if (!java.util.Arrays.equals(tag, bor.doFinal())) {
                    print("Sporočilo je bilo spremenjeno, prekinitvam!");
                    System.exit(1);
                }

                // Bor sporočilo izpiše
                print("Sporočilo se glasi: '%s'", new String(pt, StandardCharsets.UTF_8));

            }
        });

        // Povežemo Ano in Bora
        env.connect("ana", "bor");
        // zaženemo simulacijsko okolje
        env.start();
    }
}
```

### HMAC z uporabo zgoščevalne funkcije SHA3-256 (primer z shranjevanjem ključa)
**Read - Write**

#### Write

```java
public class WriteToFileHMAC {
    public static void main(String[] args) throws IOException, NoSuchAlgorithmException, InvalidKeyException {
        final Key key = KeyGenerator.getInstance("HmacSHA256").generateKey();   // Ustvarimo naključen ključ
        final String message = "Sporočilo, ki bo zapisano na disk in zavarovano."; // Ustvarimo sporočilo in ga postrojimo
        final byte[] pt = message.getBytes(StandardCharsets.UTF_8);

        final Mac mac = Mac.getInstance("HmacSHA256");  // Instanciiramo algoritem MAC
        mac.init(key);  // In ji podamo ključ
        mac.update(pt); // Dodamo čistopis
        final byte[] tag = mac.doFinal();               // Izračunamo značko

        final byte[] data = Arrays.copyOf(tag, tag.length + pt.length); // konkateniramo značko in čistopis
        System.arraycopy(pt, 0, data, tag.length, pt.length);   // Kopiramo čistopis za značko

        Files.write(Path.of("Path\\to\\msg.bin"), data); // shranimo konkatenirane podatke
        System.out.println("Data written to disk!");

        // Shranimo še ključ (na disk ga shranimo v nešifrirani obliki, kar ni varno, dovolj)
        Files.write(Path.of("Path\\to\\keyfile"), key.getEncoded());
        System.out.println("Key written to disk!");
    }
}
```

#### Read

```java
public class ReadFromFileHMAC {
    public static void main(String[] args) throws IOException, NoSuchAlgorithmException, InvalidKeyException {

        final byte[] keyBytes = Files.readAllBytes(Path.of("Path\\to\\keyfile")); // Preberemo ključ iz datoteke
        // iz podanih bajtov ustvarimo objekt Key, ki ga bomo uporabili za izračun MAC
        final Key key = new SecretKeySpec(keyBytes, "HmacSHA256");
        final Mac mac = Mac.getInstance("HmacSHA256");  // Instanciiramo algoritem MAC
        mac.init(key);  // mu podamo ključ

        final byte[] tagAndMessage = Files.readAllBytes(Path.of("Path\\to\\msg.bin"));  // Preberemo podatke iz datoteke
        final byte[] tag = Arrays.copyOfRange(tagAndMessage, 0, 32);  // Prvih 32 bajtov predstavlja značko
        final byte[] message = Arrays.copyOfRange(tagAndMessage, 32, tagAndMessage.length); // Ostali bajti so sporočilo

        mac.update(message);    // sporočilo naložimo v algoritem MAC
        final byte[] recomputedTag = mac.doFinal(); // Izračunamo značko

        if (!MessageDigest.isEqual(tag, recomputedTag)) {   // Preverimo značko
            System.out.println("Read invalid data, aborting!");
            System.exit(1);
        }
        System.out.printf("Data appears to be valid: '%s'%n", new String(message, StandardCharsets.UTF_8));
    }
}
```